# Unsupervised learning on rock properties

Sometimes we don't have labels, but would like to discover structure in a dataset. This is what clustering algorithms attempt to do. They don't require labels from us &mdash; they are 'unsupervised'.

We'll use a subset of the [Rock Property Catalog](http://subsurfwiki.org/wiki/Rock_Property_Catalog) data, licensed CC-BY Agile Scientific. Note that the data have been preprocessed, including the addition of noise to the density data.

We'll use three unsupervised clustering techniques:

- k-means clustering
- DBSCAN
- HDBSCAN

And two unsupervised dimensionality reduction techniques:

- t-SNE
- UMAP

We do have lithology labels for this dataset (normally in an unsupervised problem you wouldn't have labels), so we can use those as a measure of how well we're doing with the clustering.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('https://geocomp.s3.amazonaws.com/data/RPC_4_lithologies.csv')

In [ ]:
df.describe()

Notice that the count of `Rho` values is smaller than for the other properties.

Pairplots are a good way to see how the various features are distributed with respect to each other:

In [ ]:
features = ['Vp', 'Vs', 'Rho_n']

sns.pairplot(df.dropna(), vars=features, hue='Lithology')

## Clustering with _k_-means

From [the Wikipedia article](https://en.wikipedia.org/wiki/K-means_clustering):

> k-means clustering is a method of vector quantization, originally from signal processing, that is popular for cluster analysis in data mining. k-means clustering aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster. This results in a partitioning of the data space into Voronoi cells.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
clu = KMeans()

# This will fail...
clu.fit(df[features].values)

The old classic: NaNs. Remember the count of `Rho` points being smaller than the others?

The easiest thing to do, assuming we have the data, is to drop the rows with NaNs.

In [ ]:
df = df.dropna()

In [ ]:
clu.fit(df[features])

In [ ]:
df['Kmeans'] = clu.predict(df[features].values)

In [ ]:
for name, group in df.groupby('Kmeans'):
    plt.scatter(group.Vp, group.Rho_n, label=name)
plt.legend()

We actually do have the labels, so let's compare...

In [ ]:
for name, group in df.groupby('Lithology'):
    plt.scatter(group.Vp, group.Rho_n, label=name)
plt.legend()

In [ ]:
clu = KMeans(n_clusters=4)
df['K means'] = clu.fit_predict(df[features])
for name, group in df.groupby('K means'):
    plt.scatter(group.Vp, group.Rho_n, label=name)
plt.legend()

# We can add the centroids as well:
plt.plot(*clu.cluster_centers_[:, ::2].T, 'o', c='k', ms=10)

## Clustering with DBSCAN

DBSCAN has nothing to do with databases. From [the Wikipedia article](https://en.wikipedia.org/wiki/DBSCAN):

> Density-based spatial clustering of applications with noise (DBSCAN) is [...] a density-based clustering algorithm: given a set of points in some space, it groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away). DBSCAN is one of the most common clustering algorithms and also most cited in scientific literature.

In [ ]:
from sklearn.cluster import DBSCAN

DBSCAN()

There are two important hyperparameters:

- `eps`, the maximum distance between points in the same cluster.
- `min_samples`, the minimum number of samples in a cluster.

In [ ]:
clu = DBSCAN(eps=150, min_samples=10)

clu.fit(df[features])

In [ ]:
df['DBSCAN'] = clu.labels_

In [ ]:
for name, group in df.groupby('DBSCAN'):
    plt.scatter(group.Vp, group.Rho_n, label=name)

It's a bit hard to juggle the two parameters... let's make an interactive widget:

In [ ]:
from ipywidgets import interact

@interact(eps=(10, 250, 10))
def plot(eps):
    clu = DBSCAN(eps=eps)
    clu.fit(df[features])
    df['DBSCAN'] = clu.labels_
    for name, group in df.groupby('DBSCAN'):
        plt.scatter(group.Vp, group.Rho_n, label=name)

## HDBSCAN

### Clustering with HDBSCAN

HDBSCAN is an improvement on DBSCAN, but is not yet available in `sklearn`. Install it with:

    conda install -c conda-forge hdbscan

DBSCAN has nothing to do with databases. From [the Wikipedia article](https://en.wikipedia.org/wiki/DBSCAN):

> Density-based spatial clustering of applications with noise (DBSCAN) is [...] a density-based clustering algorithm: given a set of points in some space, it groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away). DBSCAN is one of the most common clustering algorithms and also most cited in scientific literature.

HDBSCAN does away with the epsilon parameter (the maximum distance between points in the same cluster) in DBSCAN, leaving only `n`. So there is only one important hyperparameter:

- `min_samples`, the minimum number of samples in a cluster.

In [ ]:
from hdbscan import HDBSCAN

clu = HDBSCAN(min_samples=10)

clu.fit(df[features])

df['HDBSCAN'] = clu.labels_

Anything the algothim considers to be noise is assigned -1:

In [ ]:
np.unique(clu.labels_)  # -1 is 'noise'

In [ ]:
for name, group in df.groupby('HDBSCAN'):
    plt.scatter(group.Vp, group.Rho_n, label=name)

In [ ]:
from ipywidgets import interact

@interact(min_samples=(1, 60, 2))
def plot(min_samples):
    clu = HDBSCAN(min_samples=min_samples)
    clu.fit(df[features])
    df['HDBSCAN'] = clu.labels_
    for name, group in df.groupby('HDBSCAN'):
        plt.scatter(group.Vp, group.Rho_n, label=name)

## Comparing clusterings

There are metrics for comparing clusterings. For example, `adjusted_rand_score` &mdash; from the scikit-learn docs:

> The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.
>
> The raw RI score is then “adjusted for chance” into the ARI score using the following scheme:
> 
> ARI = (RI - Expected_RI) / (max(RI) - Expected_RI)
> 
> The adjusted Rand index is thus ensured to have a value close to 0.0 for random labeling independently of the number of clusters and samples and exactly 1.0 when the clusterings are identical (up to a permutation).

Conveniently, there is no need for the labels to correspond &mdash; the algorithm just compares whether similar points in one clustering are still similar in the other

In [ ]:
from sklearn.metrics import adjusted_rand_score

print(adjusted_rand_score(df.Lithology, df.Kmeans))
print(adjusted_rand_score(df.Lithology, df.DBSCAN))

There are a lot of other clustering algorithms to try. This figure is from [the `sklearn` docs](https://scikit-learn.org/stable/modules/clustering.html):

<img src="../images/sphx_glr_plot_cluster_comparison_0011.png" />

## Manifold embedding with t-SNE

[`t-SNE`](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) was recently implemented in `sklearn`. t-statistic neighbourhood embedding is a popular and very effective dimensionality reduction strategy. The caveat is that distance between clusters is not typically meaningful. But it's at least a useful data exploration tool.

Usually we want `n_components=2`, so we get the data into a 2-space, e.g. for cross-plotting.

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(init='random', perplexity=100)

embedding = tsne.fit_transform(df[features].values)

In [ ]:
from sklearn.preprocessing import LabelEncoder

labels = LabelEncoder().fit_transform(df.Lithology)

In [ ]:
plt.scatter(*embedding.T, c=labels, cmap='tab10', vmax=10)
plt.show()

## Manifolds with UMAP

Another popular tool is UMAP. Check out [`umap-learn`](https://pypi.org/project/umap-learn/). You can install it with 

    conda install -y -c conda-forge umap-learn
    
It has the same interface as `sklearn` so it's very easy to use.

In [ ]:
from umap import UMAP

umap = UMAP(metric='euclidean')

embed_umap = umap.fit_transform(df[features].values)

In [ ]:
plt.scatter(*embed_umap.T, c=labels, cmap='tab10', vmax=10)
plt.show()

It's fairly common to attempt clustering on the result of the dimensionality reduction, but you should be wary of treating the embedding as a metric space. That is, distances in this space may not correspond to any natural or useful property.

### Exercises

- Can you make the interactive widget display the Rand score? Use `plt.text(x, y, "Text")`.
- Can you write a loop to find the value of `eps` giving the highest Rand score?
- Can you add the `min_samples` parameter to the widget?
- Explore some of [the other clustering algorithms](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster).
- Try some clustering on one of your own datasets (or use something from [sklearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets), e.g. `sklearn.datasets.load_iris`).